In [1]:
import subprocess  
import pandas as pd
import json
from os.path import exists
import distutils.spawn

In [2]:
# inputs
MODEL_NAME = 'newmodel'
TEMPLATE_NAME = 'template.apsimx'
MODELS_BIN = '/usr/local/bin/Models'

In [3]:
# checks
found = distutils.spawn.find_executable(MODELS_BIN)
if not found:
    print(f"{MODELS_BIN} not found")
if not exists(TEMPLATE_NAME):
    print(f"{TEMPLATE_NAME} not found")

In [4]:
%pwd

'/home/jmc15/Documents/juan/soy_model'

In [5]:
# load a template
template = json.load(open(TEMPLATE_NAME, 'r'))
children = template['Children']

In [7]:
# add data

In [8]:
# save new nodel
with open(f"data/{MODEL_NAME}.apsimx", 'w') as outfile:
    json.dump(template, outfile, sort_keys=True, indent=4)
# run model if Models executabl exists
result = subprocess.run(f"{MODELS_BIN} {MODEL_NAME}.apsimx",
                        shell=True, cwd='data/',
                        text=True,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
# check execution status
if result.returncode == 1:
    # error, show information
    print(f"Error: {result.stdout} {result.stderr}")
elif result.returncode == 0:
    # Ok 
    print(f"Success {result.stdout} {result.stderr}")

Success  


In [9]:
# read results
import sqlite3

con = sqlite3.connect(f"data/{MODEL_NAME}.db")
# creating cursor
cur = con.cursor()
# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
dfs = {}
for table_ in table_list:
    table = table_[0]
    dfs[table] = pd.read_sql_query(f"SELECT * FROM {table}", con)
    print(table)
# Be sure to close the connection
con.close()


_InitialConditions
_Messages
HarvestReport
_Simulations
_Checkpoints
_Units


In [16]:
# show some plots ?)

In [10]:
dfs['_Messages']

,CheckpointID,SimulationID,ComponentName,Date,Message,MessageType
0,1,1,paddock.fertiliser,1900-09-21 12:00:00,10 kg/ha of NO3N added at depth 0 layer 1,0
1,1,1,paddock.Soybean,1900-09-21 12:00:00,A crop of Soybean (cultivar = Davis) was sown ...,0
2,1,1,paddock.Soybean,1900-09-22 12:00:00,Germination\r\n LAI = 0.00 (m^2/m^2)\r\n Abo...,0
3,1,1,paddock.Soybean,1900-10-01 12:00:00,Emergence\r\n LAI = 0.00 (m^2/m^2)\r\n Above...,0
4,1,1,paddock.Soybean,1900-11-21 12:00:00,StartFlowering\r\n LAI = 6.93 (m^2/m^2)\r\n ...,0
...,...,...,...,...,...,...
2196,1,1,paddock.Soybean.Shell,2000-03-31 12:00:00,Removed 0.0 g/m2 of dry matter weight and 0.0 ...,0
2197,1,1,paddock.Soybean.Stem,2000-03-31 12:00:00,Removing 100.0% of stem biomass from Soybean. ...,0
2198,1,1,paddock.Soybean.Stem,2000-03-31 12:00:00,Removed 0.0 g/m2 of dry matter weight and 0.0 ...,0
2199,1,1,paddock.Soybean,2000-03-31 12:00:00,Crop ending,0


In [11]:
dfs['HarvestReport']

,CheckpointID,SimulationID,Zone,Clock.Today,Soybean.Phenology.Stage,Soybean.Phenology.CurrentStageName,Soybean.AboveGround.Wt,Soybean.AboveGround.N,Yield,Soybean.Grain.NConc,Soybean.Grain.Size,Soybean.Grain.Number,Soybean.Grain.Total.Wt,Soybean.Grain.Total.N,Soybean.Total.Wt
0,1,1,paddock,1901-02-21 12:00:00,11.0,HarvestRipe,576.418557,18.899368,1769.799027,0.069363,0.0,0.0,176.979903,12.275787,687.276926
1,1,1,paddock,1902-03-08 12:00:00,11.0,HarvestRipe,340.244200,10.914946,281.278337,0.075281,0.0,0.0,28.127834,2.117489,418.902172
2,1,1,paddock,1903-03-18 12:00:00,11.0,HarvestRipe,92.892317,3.678455,31.991251,0.075281,0.0,0.0,3.199125,0.240833,111.153070
3,1,1,paddock,1904-02-29 12:00:00,11.0,HarvestRipe,603.761808,18.098210,858.834854,0.074666,0.0,0.0,85.883485,6.412586,750.154977
4,1,1,paddock,1905-02-25 12:00:00,11.0,HarvestRipe,485.577171,15.804174,1414.458239,0.068242,0.0,0.0,141.445824,9.652567,587.515572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1,1,paddock,1996-03-16 12:00:00,11.0,HarvestRipe,1034.046956,33.627307,3606.011610,0.064260,0.0,0.0,360.601161,23.172259,1235.682494
84,1,1,paddock,1997-02-25 12:00:00,11.0,HarvestRipe,388.971142,13.438945,895.169099,0.066325,0.0,0.0,89.516910,5.937170,470.423444
85,1,1,paddock,1998-03-01 12:00:00,11.0,HarvestRipe,329.805082,11.814188,1122.227202,0.062893,0.0,0.0,112.222720,7.058052,392.986877
86,1,1,paddock,1999-02-09 12:00:00,11.0,HarvestRipe,553.124215,19.725954,2408.755873,0.063090,0.0,0.0,240.875587,15.196810,646.319304
